# Fine Tuning guide with OpenAI

Questo notebook Python è stato sviluppato per effettuare il fine tuning un modello di OpenAI tramite un dataset preparato precedentemente.

Il processo consiste nel:
1) Crea tre differenti file: training, validazione e test
2) Caricare sulla piattaforma i file training e validazione
3) Creazione fine tuning job
4) Verifica lo stato del job
5) Valutazione risultati

In [1]:
# Installazione dei pacchetti necessari per l'esecuzione del notebook

!pip install --quiet openai python-dotenv scikit-learn


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import dotenv, os
dotenv.load_dotenv(override=True)

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

from openai import OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)

# Training, validation & test file creation

## Input
- **File di Input:**  
  Un file `dataset.jsonl` contenente dati in formato JSONL, in cui ogni riga rappresenta un esempio.
- **Struttura dei Dati:**  
  Ogni esempio deve essere un oggetto JSON che include una chiave `messages`.  
  - `messages` è una lista di messaggi, e ogni messaggio deve avere:  
    - **role:** con valore `system`, `user` o `assistant`.
    - **content:** il testo associato al messaggio.

## Output
- **training.jsonl:**  
  Contiene il 70% degli esempi del dataset originale.
- **validation.jsonl:**  
  Contiene il 15% degli esempi.
- **test.jsonl:**  
  Contiene il restante 15% degli esempi.
- **test.csv:**  
  File CSV derivato dal test set, in cui ogni riga ha colonne separate per:
  - `system`
  - `user`
  - `assistant`

## Parametri Interni Regolabili
- **Percentuali di Partizione:**  
  - `train_ratio = 0.70` (70% per training)
  - `val_ratio = 0.15` (15% per validazione)
  - `test_ratio = 0.15` (15% per test; il test set viene calcolato come il resto degli esempi)
- **Percorsi dei File:**  
  Modifica i nomi dei file per:
  - `dataset_file` (file di input)
  - `train_file` (output training)
  - `val_file` (output validation)
  - `test_file` (output test in formato JSONL)
  - `test_csv_file` (output test in formato CSV)

  ## Note
  Il file in formato csv può essere utilizzato nel tool `Evaluation` nella platform di OpenAI https://platform.openai.com/evaluations



In [4]:
import json, csv
import random

# Configura i percorsi dei file
dataset_file = "dataset.jsonl"
train_file = "training.jsonl"
val_file = "validation.jsonl"
test_file = "test.jsonl"
test_csv_file = "test.csv"

train_ratio = 0.70  # 70% training
val_ratio = 0.15    # 15% validation
test_ratio = 0.15   # 15% test

with open(dataset_file, "r", encoding="utf-8") as f:
    data = [json.loads(line) for line in f]

# Mescola casualmente i dati
random.shuffle(data)

# Calcola gli indici per la divisione
n = len(data)
train_size = int(n * train_ratio)
val_size = int(n * val_ratio)
# Il test prende tutto il resto, per garantire che la somma sia 100%
test_size = n - train_size - val_size

# Suddividi il dataset
train_data = data[:train_size]
val_data = data[train_size:train_size+val_size]
test_data = data[train_size+val_size:]

with open(train_file, "w", encoding="utf-8") as f:
    for entry in train_data:
        f.write(json.dumps(entry, ensure_ascii=False) + "\n")

with open(val_file, "w", encoding="utf-8") as f:
    for entry in val_data:
        f.write(json.dumps(entry, ensure_ascii=False) + "\n")

with open(test_file, "w", encoding="utf-8") as f:
    for entry in test_data:
        f.write(json.dumps(entry, ensure_ascii=False) + "\n")

def extract_messages(message_list):
  """
  Estrae i messaggi di ruolo e contenuto da una lista di messaggi
  """
  roles = {"system": "", "user": "", "assistant": ""}
  for message in message_list:
      role = message.get("role")
      if role in roles:
          roles[role] = message.get("content", "")
  return roles

with open(test_csv_file, "w", newline="", encoding="utf-8") as csvfile:
    fieldnames = ["system", "user", "assistant"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for entry in test_data:
        roles = extract_messages(entry.get("messages", []))
        writer.writerow(roles)

print(f"✅ Creazione completata")
print(f"\t📄{train_file} ({len(train_data)} esempi)")
print(f"\t📄{val_file} ({len(val_data)} esempi)")
print(f"\t📄{test_file} e {test_csv_file} ({len(test_data)} esempi).")

✅ Creazione completata
	📄training.jsonl (32 esempi)
	📄validation.jsonl (6 esempi)
	📄test.jsonl e test.csv (8 esempi).


# Upload training and validation files

In [4]:
# Upload a training file

def upload_file(file_name: str, purpose: str) -> str:
    with open(file_name, "rb") as file_fd:
        response = client.files.create(file=file_fd, purpose=purpose)
    return response.id

training_file_id = upload_file(train_file, "fine-tune")
validation_file_id = upload_file(val_file, "fine-tune")

print("Training file ID:", training_file_id)
print("Validation file ID:", validation_file_id)

Training file ID: file-BW8HoFupjBLj4f2NNQ4DSs
Validation file ID: file-TEB8YqBTryZgHKRxUnVngn


# Create FT job

In [ ]:
MODEL = "gpt-4o-mini-2024-07-18"

job = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model=MODEL,
    # method={
    #   "type": "supervised",
    #   "supervised": {
    #     "hyperparameters": {
    #       "n_epochs": 2,
    #       "batch_size": 10,
    #       "learning_rate_multiplier": 1.0
    #     }
    #   }
    # }
)

# print(job)
print("Job ID:", job.id)


FineTuningJob(id='ftjob-5UYBh40uYtHVzr5iTLY0Uo3g', created_at=1739808524, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-eF6QgROkc7s5rEEip6fS8fjY', result_files=[], seed=994339359, status='validating_files', trained_tokens=None, training_file='file-BW8HoFupjBLj4f2NNQ4DSs', validation_file='file-TEB8YqBTryZgHKRxUnVngn', estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto')), type='supervised'), user_provided_suffix=None)


# Check job status

In [10]:
# Work with fine-tuning jobs

print("List 10 fine-tuning jobs")
print(client.fine_tuning.jobs.list(limit=10))

print("=" * 50)

print("Retrieve the state of a fine-tune")
response = client.fine_tuning.jobs.retrieve(job.id)

print("Job ID:", response.id)
print("Status:", response.status)
fine_tuned_model_id = response.fine_tuned_model

if fine_tuned_model_id is None:
    print("Fine-tuned model ID not found. Your job has likely not been completed yet.")
else:
    print("Fine-tuned model ID:", fine_tuned_model_id)

# Cancel a job
#client.fine_tuning.jobs.cancel(fine_tuned_model_id)

# List up to 10 events from a fine-tuning job
#client.fine_tuning.jobs.list_events(fine_tuning_job_id=fine_tuned_model_id, limit=10)

# Delete a fine-tuned model
#client.models.delete(fine_tuned_model_id)

List 10 fine-tuning jobs
SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-5UYBh40uYtHVzr5iTLY0Uo3g', created_at=1739808524, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:rsn-labcamp::B1y5EZ1L', finished_at=1739808987, hyperparameters=Hyperparameters(batch_size=1, learning_rate_multiplier=1.8, n_epochs=3), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-eF6QgROkc7s5rEEip6fS8fjY', result_files=['file-3CcGBNactwKvBo3xSVuGM4'], seed=994339359, status='succeeded', trained_tokens=13914, training_file='file-BW8HoFupjBLj4f2NNQ4DSs', validation_file='file-TEB8YqBTryZgHKRxUnVngn', estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size=1, learning_rate_multiplier=1.8, n_epochs=3)), type='supervised'), user_provided_suffix=None), FineTuningJob(id='ftjob-bFNIrGhRuYsjwELebRog3oOk', created_at=1739655494, error=Error(

# Evaluation

Tramite la cosine similarity tra gli embeddings della risposta desiderata vs quella ottenuta, si calcola la distanza tra le risposte così da valutare il fine tuning: più è alto il risultato, maggiore è somiglianza/vicinanza.

Tramite le variabili di seguito è possibile regolare:

- `N_EXAMPLES`: numero di esempi del test dataset da valutare
- `embeddings_model`: modello di OpenAI per il calcolo degli embeddings

In [ ]:
import json
from sklearn.metrics.pairwise import cosine_similarity

# Legge i primi N esempi dal file test.jsonl

N_EXAMPLES=10
embeddings_model="text-embedding-3-small"

examples = []
with open(test_file, "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        if i >= N_EXAMPLES:
            break
        examples.append(json.loads(line))

def get_embedding(text, model=embeddings_model):
    response = client.embeddings.create(input=text, model=model)
    return response.data[0].embedding

def cosine_sim(vec1, vec2):
    return cosine_similarity([vec1], [vec2])[0][0]

def get_model_response(prompt):
    completion = client.chat.completions.create(
        model=fine_tuned_model_id,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    return completion.choices[0].message.content

total_similarity = 0
valid_examples = 0

for i, example in enumerate(examples):
    messages = example["messages"]
    user_message = next((msg["content"] for msg in messages if msg["role"] == "user"), None)
    reference_response = next((msg["content"] for msg in messages if msg["role"] == "assistant"), None)

    if user_message is None or reference_response is None:
        print(f"Esempio {i} non valido: manca un messaggio user o assistant")
        continue

    model_response = get_model_response(user_message)

    ref_embedding = get_embedding(reference_response)
    model_embedding = get_embedding(model_response)

    similarity = cosine_sim(ref_embedding, model_embedding)

    total_similarity += similarity
    valid_examples += 1

    print(f"Esempio {i+1} - Similarità: {similarity:.4f}")
    print("Input:", user_message)
    print("Risposta di riferimento:", reference_response)
    print("Risposta del modello:", model_response)
    print("-" * 50)

print("-" * 50 + "\n")
if valid_examples > 0:
    average_similarity = total_similarity / valid_examples
    print(f"Similarità media: {average_similarity:.4f}")
else:
    print("❌ Nessun esempio valido per il calcolo della similarità media.")

Esempio 1 - Similarità: 0.8978
Input: Quali sono le date del Festival di Sanremo 2024?
Risposta di riferimento: Il settantaquattresimo Festival di Sanremo si è svolto dal 6 al 10 febbraio 2024.
Risposta del modello: Il Festival di Sanremo 2024 si è svolto dal 6 al 10 febbraio 2024.
--------------------------------------------------
Esempio 2 - Similarità: 0.9080
Input: Quale artista ha vinto il Festival di Sanremo 2024?
Risposta di riferimento: Angelina Mango è stata la vincitrice del Festival di Sanremo 2024 con il brano 'La noia'.
Risposta del modello: Il Festival di Sanremo 2024 è stato vinto da Angelina Mango con il brano 'La noia'.
--------------------------------------------------
Esempio 3 - Similarità: 1.0000
Input: Quanti artisti hanno partecipato al Festival di Sanremo 2024?
Risposta di riferimento: Al Festival di Sanremo 2024 hanno partecipato 30 artisti.
Risposta del modello: Al Festival di Sanremo 2024 hanno partecipato 30 artisti.
-----------------------------------------